### 0-导语
每一个算法——简单的概括介绍（功能运用/原理概述）+代码示例

### 10-使用 Keras 搭建人工神经网络
本章的第一部分会介绍人工神经网络，从一个简单的 ANN 架构开始，然后过渡到多层感知机（MLP）。第二部分会介绍如何使用流行的 Keras API 搭建神经网络，Keras API 是一个高级 API，可以用来搭建、训练、评估、运行神经网络。Keras 的易用性，并不妨碍它具有强大的实现能力，Keras 足以帮你搭建多种多样的神经网络。事实上，Keras 足以完成大多数的任务啦！要是你需要实现更多的功能，你可以用 Keras 的低级 API（第 12 章介绍）自己写一些组件。

感知机的学习算法（权重更新算法）与随机梯度下降基本一致，具有线性分类模型的局限性。
感知机的一些局限性可以通过堆叠多个感知机消除。由此产生的人工神经网络被称为多层感知机（MLP）
使用反向传播算法/BP算法优化参数。

In [ ]:
# 调用tensorflow、keras
import tensorflow as tf
from tensorflow import keras

tf.__version__ , keras.__version__

回归MLP —— 典型架构总结
<img src='图库/C10/86424dc7bed03e26a159be77a1f3fc8a.png'><br>

分类MLP —— 典型架构总结
<img src='图库/C10/a448b0d1e942484bff4a067c11441c2e.png'><br>

In [ ]:
# 使用 Keras 加载数据集
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data() 

# 查看数据
X_train_full.shape, X_train_full.dtype
# 对于 Fashion MNIST，需要分类名的列表：
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

# 创建验证集：该数据集已经分成了训练集和测试集，但没有验证集。
# 特征缩放：简单起见，通过除以255.0将强度范围变为0-1
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:] 

step1-用顺序 API 创建模型

In [ ]:
# 传递一个层组成的列表：
model = keras.models.Sequential([
    # 接下来创建了第一层，这是一个Flatten层，它的作用是将每个输入图片转变为 1D 数组：
    keras.layers.Flatten(input_shape=[28, 28]),
    # 然后，添加了一个有 300 个神经元的紧密层，激活函数是 ReLU。
    keras.layers.Dense(300, activation="relu"),
    # 接着再添加第二个紧密层，激活函数仍然是 ReLU
    keras.layers.Dense(100, activation="relu"),
    # 最后，加上一个拥有 10 个神经元的输出层（每有一个类就要有一个神经元）
    keras.layers.Dense(10, activation="softmax")
]) 

# 模型的summary()方法可以展示所有层，包括每个层的名字（名字是自动生成的，除非建层时指定名字），
# 输出的形状（None代表批次大小可以是任意值），和参数的数量。
# 最后会输出所有参数的数量，包括可训练和不可训练参数。
model.summary()

In [ ]:
# 使用属性，获取神经层很容易，可以通过索引或名称获取对应的层：
model.layers

hidden1 = model.layers[1]
hidden1.name

model.get_layer('dense_3') is hidden1

In [ ]:
# 可以用get_weights()和set_weights()方法，获取神经层的所有参数。对于紧密层，参数包括连接权重和偏置项：
hidden1 = model.layers[1]
weights, biases = hidden1.get_weights()

#### 3-5保存和恢复模型
使用顺序 API 或函数式 API 时，保存训练好的 Keras 模型和训练一样简单：<br>
(警告：这种加载模型的方法只对顺序 API 或函数式 API 有用，不适用于子类化 API。对于后者，可以用save_weights()和load_weights()保存参数，其它的就得手动保存恢复了。)
Keras 使用 HDF5 格式保存模型架构（包括每层的超参数）和每层的所有参数值（连接权重和偏置项）。还保存了优化器（包括超参数和状态）。<br>

In [ ]:
# 保存模型
model.save("my_keras_model.h5") 

# 恢复模型
import h5py
model_h5 = h5py.File("my_keras_model.h5")
loaded_model = keras.models.load_model(model_h5) 

调回callbacks——保存检查点、早停

——保存检查点: ModelCheckpoint可以在每个时间间隔保存检查点，默认是每个周期结束之后<br>
——早停：使用EarlyStopping。当检测到经过几个周期（周期数由参数patience确定），验证集表现没有提升时，就会中断训练，还能自动回到最优模型。<br>
可以将保存检查点（避免宕机）和早停（避免浪费时间和资源）结合起来。(提示：包keras.callbacks中还有其它可用的调回。)<br>

In [ ]:
# 模型保存为 my_keras_model.h5 文件
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5")
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)

# fit()方法接受参数callbacks，可以让用户指明一个 Keras 列表，让 Keras 在训练过程中调用保存数据。
history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb]) 

编写自定义的调回

In [ ]:
# 下面的例子展示了一个可以展示验证集损失和训练集损失比例的自定义（检测过拟合）调回
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"])) 

问题：
批次样本量如何设置？

In [ ]:
# 以下代码应用 PCA 将数据集的维度降至两维（请注意，它会自动处理数据的中心化）
from sklearn.decomposition import PCA

# 维度/方差比率
pca=PCA(n_components=2/0.95)
X_reduced=pca.fit_transform(X)

### 11-训练深度神经网络
第 10 章介绍了人工神经网络，并训练了第一个深度神经网络。 但它非常浅，只有两个隐藏层。 如果你需要解决非常复杂的问题，例如检测高分辨率图像中的数百种类型的对象，该怎么办？ 你可能需要训练更深的 DNN，也许有 10 层或更多，每层包含数百个神经元，通过数十万个连接相连。 这可不像公园散步那么简单，可能碰到下面这些问题：

你将面临棘手的梯度消失问题（或相关的梯度爆炸问题）：在反向传播过程中，梯度变得越来越小或越来越大。二者都会使较浅层难以训练；
要训练一个庞大的神经网络，但是数据量不足，或者标注成本很高；
训练可能非常慢；
具有数百万参数的模型将会有严重的过拟合训练集的风险，特别是在训练实例不多或存在噪音时。
在本章中，我们将依次讨论这些问题，并给出解决问题的方法。 我们将从梯度消失/爆炸问题开始，并探讨解决这个问题的一些最流行的解决方案。 接下来会介绍迁移学习和无监督预训练，这可以在即使标注数据不多的情况下，也能应对复杂问题。然后我们将看看各种优化器，可以加速大型模型的训练。 最后，我们将浏览一些流行的大型神经网络正则化方法。

加速训练/收敛训练深度神经网络：
针对梯度消失/爆炸问题：对连接权重应用良好的初始化策略，使用良好的激活函数，使用批归一化
重用预训练网络：迁移学习、无监督学习（针对标签少）

#### 1-梯度消失/爆炸问题
问题描述：随着算法进展到较低层，梯度往往变得越来越小。 结果，梯度下降更新使得低层连接权重实际上保持不变，并且训练永远不会收敛到最优解。 这被称为梯度消失问题。 在某些情况下，可能会发生相反的情况：梯度可能变得越来越大，许多层得到了非常大的权重更新，算法发散。这是梯度爆炸的问题，在循环神经网络中最为常见

原因：
根本原因：层输出方差相较于输入方差会显著增大/减小，且层输出均值相较于输入均值也会产生偏移。
用这个激活函数和这个初始化方案（sigmoid 激活函数和当时最受欢迎的权重初始化方法的组合，即随机初始化时使用平均值为 0，标准差为 1 的正态分布。），每层输出的方差远大于其输入的方差。随着网络前向传播，每层的方差持续增加，直到激活函数在顶层饱和。
函数饱和，梯度/导数非常接近 0。

初始化

Glorot 和 He 初始化
Xavier 初始化/ Glorot 初始化——针对逻辑激活函数

激活函数

非饱和激活函数(ReLU)>饱和激活函数
“ReLU 死区” ——> ReLU变体
绘制表格比较

批归一化（Batch Normalization）
每个批归一化的层都通过指数移动平均学习了四个参数：γ（输出缩放向量），β（输出偏移向量），μ（最终输入平均值向量）和σ（最终输入标准差向量）。μ和σ都是在训练过程中计算的，但只在训练后使用（用于替换公式 11-3 中批输入平均和标准差）。